In [1]:
import numpy as np
import requests as rq
import pandas as pd

from nba_api.stats.endpoints import playercareerstats
from nba_api.live.nba.endpoints import scoreboard
from nba_api.stats.endpoints import leagueleaders
from nba_api.stats.endpoints import playergamelog
from nba_api.stats.endpoints import commonteamroster
from nba_api.stats.endpoints import leaguestandings
from nba_api.stats.static import teams
from nba_api.stats.endpoints import leaguedashteamstats
from nba_api.stats.endpoints import leaguedashteamstats

Games of the Day

In [2]:

games = scoreboard.ScoreBoard().get_dict()['scoreboard']['games']
columns = ['HomeTeamID', 'Home', 
           'AwayTeamID', 'Away']
games_today = pd.DataFrame(index=range(len(games)), columns = columns)

for i in range(len(games)):
    games_today['HomeTeamID'][i] = games[i]['homeTeam']['teamId']
    games_today['AwayTeamID'][i] = games[i]['awayTeam']['teamId']
    games_today['Home'][i] = games[i]['homeTeam']['teamTricode']
    games_today['Away'][i] = games[i]['awayTeam']['teamTricode']
    # games_today['HomeTeamWins'][i] = games[i]['homeTeam']['wins']
    # games_today['AwayTeamWins'][i] = games[i]['awayTeam']['wins']
    # games_today['HomeTeamLosses'][i] = games[i]['homeTeam']['losses']
    # games_today['AwayTeamLosses'][i] = games[i]['awayTeam']['losses']

games_today.to_csv('games_today.csv', index=False)



league_rankings will show the ranking of each major stat for all teams in the NBA

In [3]:
league_rankings = leaguedashteamstats.LeagueDashTeamStats().get_data_frames()[0][['TEAM_ID', 
        'TEAM_NAME', 'GP', 'W', 'L', 'REB_RANK', 'AST_RANK', 'TOV_RANK', 'STL_RANK', 'BLK_RANK',
       'BLKA_RANK', 'PF_RANK', 'PFD_RANK', 'PTS_RANK', 'PLUS_MINUS_RANK']]
league_rankings.to_csv('league_rankings.csv', index=False)

Takes the games_today and creates a HOME and AWAY dataframe for all the players today and then put them into a single data frame.

In [4]:

home_teams = games_today['HomeTeamID'].to_list()
away_teams = games_today['AwayTeamID'].to_list()

home_opp = []
away_opp = []
home_away = []
away_home = []

columns = ['TeamID', 'PLAYER', 'POSITION', 'PLAYER_ID']
home_rosters = pd.DataFrame(columns = columns)
away_rosters = pd.DataFrame(columns = columns)

for team in range(len(home_teams)):
    home_rosters = pd.concat([home_rosters, commonteamroster.CommonTeamRoster(team_id=f"{home_teams[team]}").get_data_frames()[0][columns]], ignore_index=True)
    

for team in range(len(away_teams)):
    away_rosters = pd.concat([away_rosters, commonteamroster.CommonTeamRoster(team_id=f"{away_teams[team]}").get_data_frames()[0][columns]], ignore_index=True)
    

for player in range(len(home_rosters)):
    for i in range(len(home_teams)):
        if games_today['HomeTeamID'][i] == home_rosters['TeamID'][player]:
            home_opp.append(games_today['Away'][i])
            home_away.append('Home')
          
for player in range(len(away_rosters)):
    for i in range(len(away_teams)):
        if games_today['AwayTeamID'][i] == away_rosters['TeamID'][player]:
            away_opp.append(games_today['Home'][i])     
            away_home.append('Away')
            
home_opp = pd.DataFrame(home_opp, columns=['OPP'])
home_away = pd.DataFrame(home_away, columns=['HOME/AWAY'])
away_home = pd.DataFrame(away_home, columns=['HOME/AWAY'])

home_rosters = pd.concat([home_rosters, home_opp], axis=1)
home_rosters = pd.concat([home_rosters, home_away], axis=1)

away_opp = pd.DataFrame(away_opp, columns=['OPP'])
away_rosters = pd.concat([away_rosters, away_opp], axis=1)
away_rosters = pd.concat([away_rosters, away_home], axis=1)


players_today = pd.concat([home_rosters, away_rosters], axis=0)
players_today.reset_index(drop=True, inplace=True)
## Save this will get home/away rosters
players_today.to_csv('players_today.csv', index=False)

The following takes from the players playing today and creates a CSV of each player's career stats. 

In [5]:
players_today_matchups = pd.DataFrame()
for player in range(len(players_today)):
# for player in range(12,20):

    matchup = playergamelog.PlayerGameLog(player_id=players_today['PLAYER_ID'][player]).get_data_frames()[0]
    result = matchup[matchup['MATCHUP'].str.contains(players_today['OPP'][player])]
    players_today_matchups = pd.concat([players_today_matchups, result], axis=0)
players_today_matchups.to_csv('player_matchup_stats.csv', index=False)

Below will take all the players that are home and create a data frame of both their stats at home as well as their last 10 games

In [16]:
## home/away stats

home_stats_df = pd.DataFrame()
away_stats_df = pd.DataFrame()
last_ten_df = pd.DataFrame()
# last_ten = pd.DataFrame()
GP_home = []
GP_away = []

stat_columns = ['FG3M', 'REB', 'AST', 'STL', 'BLK', 'PTS']
# for player in range(12,14):
for player in range(len(home_rosters)):
    # print(player)
    ## game_log will get the player throughout the season
    game_log = playergamelog.PlayerGameLog(player_id=players_today['PLAYER_ID'][player]).get_data_frames()[0]
   

       ## home and away statistics
    home_games = game_log[~game_log['MATCHUP'].str.contains('@')]
    away_games = game_log[game_log['MATCHUP'].str.contains('@')]
    last_ten = game_log[stat_columns].head(10)
    last_ten.insert(0, 'PLAYER_ID', players_today['PLAYER_ID'][player])
    
    try:
        home_stats = round(home_games[stat_columns].sum()/len(home_games),2)
        away_stats = round(away_games[stat_columns].sum()/len(home_games),2)

        # print(home_stats)
        # last_10_stats = last_ten[['FG3M', 'REB', 'AST', 'STL', 'BLK', 'PTS']].sum()/10
    except:
        home_stats = pd.Series([0, 0, 0, 0, 0, 0], index= stat_columns)
        away_stats = pd.Series([0, 0, 0, 0, 0, 0], index= stat_columns)

        print(player)
        # last_10_stats = pd.Series([0, 0, 0, 0, 0, 0], index= stat_columns)
    
    home_stats_df = pd.concat([home_stats_df, home_stats.to_frame().transpose()], ignore_index=True )
    away_stats_df = pd.concat([away_stats_df, away_stats.to_frame().transpose()], ignore_index=True )

    last_ten_df = pd.concat([last_ten_df, last_ten], ignore_index=True )
    GP_home.append(len(home_games))
    GP_away.append(len(away_games))

home_stats_df.insert(0, 'GP', GP_home)
away_stats_df.insert(0, 'GP', GP_away)



# Rename column headers
# new_column_names = {'FG3M': 'L10-FG3M', 'REB': 'L10-REB','AST': 'L10-AST', 'STL': 'L10-STL','BLK': 'L10-BLK', 'PTS': 'L10-PTS'}
# last_ten_df = last_ten_df.rename(columns=new_column_names)
    
# player_stats = pd.concat([home_rosters,home_stats_df,last_ten_df], axis=1)
home_stats_df.to_csv('home_player_stats.csv', index=False)
away_stats_df.to_csv('away_player_stats.csv', index=False)
last_ten_df.to_csv('last_ten_games.csv', index=False)
## Save pulls the stats for each home player

9
34
51
53
68
85


In [7]:
## player id --> create a loop for this to grap the current year stats
## season averages

career_stat_columns = ['FG3M', 'REB', 'AST', 'STL', 'BLK', 'PTS']
careers = pd.DataFrame()
for i in range(len(players_today)):
# for i in range(2,5):

    career = playercareerstats.PlayerCareerStats(players_today['PLAYER_ID'][i]).get_data_frames()[0]
    # print(home_rosters['PLAYER_ID'][i])
    # Divide each selected column by the number of games played
    per_game_averages = career[career_stat_columns].div(career['GP'], axis=0)
    # print(per_game_averages)
    per_game_averages.insert(0, 'PLAYER_ID', career['PLAYER_ID'])
    per_game_averages.insert(0, 'SEASON_ID', career['SEASON_ID'])
    per_game_averages.insert(2, 'TEAM_ID', career['TEAM_ID'])
    per_game_averages.insert(3, 'TEAM', career['TEAM_ABBREVIATION'])

    careers = pd.concat([careers, per_game_averages.tail(1)], axis=0, ignore_index=True)

careers.to_csv('home_players_careers.csv', index=False)


In [8]:
careers.to_csv('year_stats.csv', index=False)


In [20]:
## All teams and their standings
league_standings = leaguestandings.LeagueStandings().get_data_frames()[0]
league_standings

,LeagueID,SeasonID,TeamID,TeamCity,TeamName,Conference,ConferenceRecord,PlayoffRank,ClinchIndicator,Division,...,May,Jun,Jul,Aug,Sep,Oct,Nov,Dec,PreAS,PostAS
0,00,22023,1610612738,Boston,Celtics,East,28-6,1,,Atlantic,...,None,None,None,None,None,3-0,11-4,12-2,41-12,None
1,00,22023,1610612750,Minnesota,Timberwolves,West,25-8,1,,Northwest,...,None,None,None,None,None,1-2,13-2,10-3,37-16,None
2,00,22023,1610612739,Cleveland,Cavaliers,East,23-11,2,,Central,...,None,None,None,None,None,1-3,9-6,8-5,35-17,None
3,00,22023,1610612760,Oklahoma City,Thunder,West,22-13,2,,Northwest,...,None,None,None,None,None,3-1,9-5,10-3,36-17,None
4,00,22023,1610612746,LA,Clippers,West,21-13,3,,Pacific,...,None,None,None,None,None,3-1,5-9,11-2,35-17,None
5,00,22023,1610612749,Milwaukee,Bucks,East,25-12,3,,Central,...,None,None,None,None,None,2-1,11-5,11-2,35-19,None
6,00,22023,1610612743,Denver,Nuggets,West,20-12,4,,Northwest,...,None,None,None,None,None,4-0,9-6,10-5,36-18,None
7,00,22023,1610612752,New York,Knicks,East,22-11,4,,Atlantic,...,None,None,None,None,None,2-2,9-5,6-8,33-21,None
8,00,22023,1610612740,New Orleans,Pelicans,West,23-18,5,,Southwest,...,None,None,None,None,None,2-1,8-8,9-5,32-22,None
9,00,22023,1610612755,Philadelphia,76ers,East,21-13,5,,Atlantic,...,None,None,None,None,None,2-1,10-5,10-4,32-21,None


BELOW IS STORAGE FOR PREVIOUS CODE THAT HAS NOW BEEN UPDATED

In [73]:
from nba_api.stats.endpoints import teamdashboardbygeneralsplits
teamdashboardbygeneralsplits.TeamDashboardByGeneralSplits()

TypeError: TeamDashboardByGeneralSplits.__init__() missing 1 required positional argument: 'team_id'

In [69]:
## all nba teams

# get_teams returns a list of 30 dictionaries, each an NBA team.
nba_teams = teams.get_teams()
all_teams = pd.DataFrame(nba_teams)
all_teams

,id,full_name,abbreviation,nickname,city,state,year_founded
0,1610612737,Atlanta Hawks,ATL,Hawks,Atlanta,Georgia,1949
1,1610612738,Boston Celtics,BOS,Celtics,Boston,Massachusetts,1946
2,1610612739,Cleveland Cavaliers,CLE,Cavaliers,Cleveland,Ohio,1970
3,1610612740,New Orleans Pelicans,NOP,Pelicans,New Orleans,Louisiana,2002
4,1610612741,Chicago Bulls,CHI,Bulls,Chicago,Illinois,1966
5,1610612742,Dallas Mavericks,DAL,Mavericks,Dallas,Texas,1980
6,1610612743,Denver Nuggets,DEN,Nuggets,Denver,Colorado,1976
7,1610612744,Golden State Warriors,GSW,Warriors,Golden State,California,1946
8,1610612745,Houston Rockets,HOU,Rockets,Houston,Texas,1967
9,1610612746,Los Angeles Clippers,LAC,Clippers,Los Angeles,California,1970


In [72]:
from nba_api.stats.endpoints import teamdashboardbygeneralsplits
n = pd.DataFrame()
for team in range(len(all_teams)):
# for team in range(0,3):
    # print(team)
    new = teamdashboardbygeneralsplits.TeamDashboardByGeneralSplits(all_teams['id'].to_list()[team]).get_data_frames()[0]
    # print(new)
    n = pd.concat([n, new], axis=0)
    # print(n)
n

,GROUP_SET,GROUP_VALUE,SEASON_YEAR,GP,W,L,W_PCT,MIN,FGM,FGA,...,REB_RANK,AST_RANK,TOV_RANK,STL_RANK,BLK_RANK,BLKA_RANK,PF_RANK,PFD_RANK,PTS_RANK,PLUS_MINUS_RANK
0,Overall,2023-24,2023-24,58,26,32,0.448,2798.75,2525,5424,...,1,1,1,1,1,1,1,1,1,1
0,Overall,2023-24,2023-24,58,46,12,0.793,2809.00,2521,5205,...,1,1,1,1,1,1,1,1,1,1
0,Overall,2023-24,2023-24,57,38,19,0.667,2746.00,2426,5014,...,1,1,1,1,1,1,1,1,1,1
0,Overall,2023-24,2023-24,59,35,24,0.593,2837.00,2528,5194,...,1,1,1,1,1,1,1,1,1,1
0,Overall,2023-24,2023-24,58,27,31,0.466,2824.00,2416,5191,...,1,1,1,1,1,1,1,1,1,1
0,Overall,2023-24,2023-24,58,33,25,0.569,2784.00,2494,5205,...,1,1,1,1,1,1,1,1,1,1
0,Overall,2023-24,2023-24,58,39,19,0.672,2784.00,2537,5157,...,1,1,1,1,1,1,1,1,1,1
0,Overall,2023-24,2023-24,57,30,27,0.526,2766.00,2481,5242,...,1,1,1,1,1,1,1,1,1,1
0,Overall,2023-24,2023-24,58,25,33,0.431,2809.00,2392,5224,...,1,1,1,1,1,1,1,1,1,1
0,Overall,2023-24,2023-24,56,37,19,0.661,2693.00,2400,4881,...,1,1,1,1,1,1,1,1,1,1


In [104]:
from nba_api.stats.endpoints import leaguedashteamstats
offense_team_stats = leaguedashteamstats.LeagueDashTeamStats().get_data_frames()[0]
new.columns

Index(['TEAM_ID', 'TEAM_NAME', 'GP', 'W', 'L', 'W_PCT', 'MIN', 'FGM', 'FGA',
       'FG_PCT', 'FG3M', 'FG3A', 'FG3_PCT', 'FTM', 'FTA', 'FT_PCT', 'OREB',
       'DREB', 'REB', 'AST', 'TOV', 'STL', 'BLK', 'BLKA', 'PF', 'PFD', 'PTS',
       'PLUS_MINUS', 'GP_RANK', 'W_RANK', 'L_RANK', 'W_PCT_RANK', 'MIN_RANK',
       'FGM_RANK', 'FGA_RANK', 'FG_PCT_RANK', 'FG3M_RANK', 'FG3A_RANK',
       'FG3_PCT_RANK', 'FTM_RANK', 'FTA_RANK', 'FT_PCT_RANK', 'OREB_RANK',
       'DREB_RANK', 'REB_RANK', 'AST_RANK', 'TOV_RANK', 'STL_RANK', 'BLK_RANK',
       'BLKA_RANK', 'PF_RANK', 'PFD_RANK', 'PTS_RANK', 'PLUS_MINUS_RANK'],
      dtype='object')

In [105]:
offense_team_stats['PFD_RANK']

0      5
1     26
2     29
3     30
4     18
5     23
6      2
7     20
8     24
9     19
10    10
11    16
12    17
13     4
14    13
15    12
16     6
17     7
18    14
19     9
20    15
21     1
22    11
23     3
24    27
25    21
26    22
27    25
28     8
29    28
Name: PFD_RANK, dtype: int64

In [107]:
from nba_api.stats.endpoints import playerdashboardbyteamperformance
n = playerdashboardbyteamperformance.PlayerDashboardByTeamPerformance().get_data_frames()

TypeError: PlayerDashboardByTeamPerformance.__init__() missing 1 required positional argument: 'player_id'

In [108]:
from nba_api.stats.endpoints import leaguedashptteamdefend
n = leaguedashptteamdefend.LeagueDashPtTeamDefend().get_data_frames()

In [66]:
from nba_api.stats.endpoints import leaguedashteamstats
n = leaguedashteamstats.LeagueDashTeamStats().get_data_frames()[0]

In [67]:
n

,TEAM_ID,TEAM_NAME,GP,W,L,W_PCT,MIN,FGM,FGA,FG_PCT,...,REB_RANK,AST_RANK,TOV_RANK,STL_RANK,BLK_RANK,BLKA_RANK,PF_RANK,PFD_RANK,PTS_RANK,PLUS_MINUS_RANK
0,1610612737,Atlanta Hawks,58,26,32,0.448,2798.75,2525,5424,0.466,...,7,18,14,13,25,22,11,4,4,21
1,1610612738,Boston Celtics,58,46,12,0.793,2809.00,2521,5205,0.484,...,2,19,4,29,2,1,2,26,5,1
2,1610612751,Brooklyn Nets,58,22,36,0.379,2799.00,2400,5255,0.457,...,9,15,8,25,15,13,15,29,25,23
3,1610612766,Charlotte Hornets,58,15,43,0.259,2794.00,2345,5126,0.457,...,28,25,19,26,22,17,13,30,28,30
4,1610612741,Chicago Bulls,58,27,31,0.466,2824.00,2416,5191,0.465,...,18,28,2,8,16,15,14,20,26,20
5,1610612739,Cleveland Cavaliers,57,38,19,0.667,2746.00,2426,5014,0.484,...,12,13,13,20,21,10,5,15,24,4
6,1610612742,Dallas Mavericks,58,33,25,0.569,2784.00,2494,5205,0.479,...,23,21,3,27,23,2,12,2,8,15
7,1610612743,Denver Nuggets,58,39,19,0.672,2784.00,2537,5157,0.492,...,10,4,6,24,11,8,10,23,15,9
8,1610612765,Detroit Pistons,58,9,49,0.155,2799.00,2446,5164,0.474,...,16,16,29,30,19,26,29,21,23,26
9,1610612744,Golden State Warriors,57,30,27,0.526,2766.00,2481,5242,0.473,...,5,7,22,28,27,11,22,18,11,12
